# Imports

In [150]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

from scipy.signal import savgol_filter

In [4]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 21})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Choose and load data

In [7]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

address_runs = [x for x in all_subdir if 'DC01_RF08' in x]
print(address_runs)

> Répertoire : I:/Universite_Simus/20201201/DC07_RF09/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 19, 28, 38]
> slashcond | -2
No points deleted because they were not complete
Total number of data directories 1
[]


In [8]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC07_RF09']
000 > DC07_RF09
> condition names ['DC', 'RF']
> number of points 1
> N_ions = 1024
> e_GMol = 50


In [9]:
## Data loading (1D arrays)
# select data with init to reuse
forcetryinit = ['Try00']

data_name, num_runs \
= data_retrieve_RFRelax(all_subdir,points_and_coord,condition_parameters,slash_cfg,
                        mode='RFrelax', forcecondnum = 0, forcetryinit = forcetryinit) # stopping_power

data0 = data_name[1]
data6 = data_name[2]

> Points | 1
> Simulations pour chaque point | ['Try00']
Hello
['Temp_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.dat']
0
Point n° 0
00 - 00  >  I:/Universite_Simus/20201201\DC07_RF09\Try00\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
Time elapsed:  0.03191049999998086 s
Time elapsed:  0.0005318416666663476 m


# load context variables

In [10]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [11]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

['I:/Universite_Simus/20201201\\DC07_RF09\\Try00\\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG']/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [258]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)
plot_try = 1
address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(all_subdir[plot_point]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(all_subdir[0]+'/'+'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_init = tmp_info[-3]
i_laser_fly_quench = tmp_info[-2]
i_relax_fly = tmp_info[-7]

n_dt = 100
dt = 1/2e6

print('eta',eta)
print('i_free__fly_Lan',i_free__fly_Lan)
print('i_laser_fly_quench',i_laser_fly_init)
print('i_laser_fly_quench',i_laser_fly_quench)
print('i_relax_fly',i_relax_fly)

I:/Universite_Simus/20201201\DC07_RF09\Try00
['Langevin_cooling.info'
 'xva_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info'
 'xva_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG.info']
eta 1.5e-20
i_free__fly_Lan 1000.0
i_laser_fly_quench 6000.0
i_laser_fly_quench 10000.0
i_relax_fly 75000.0


In [206]:
str_load = work_rep + '/xNa_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG'

fid = open(str_load + '.bin', 'rb')

junk0 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk1 = fromfile(fid, int32, 1)  # Read record stop tag
jmax = aux[0]

junk2 = fromfile(fid, int32, 1)  # Read record start tag
aux = fromfile(fid, int32, 1)
junk3 = fromfile(fid, int32, 1)  # Read record stop tag
N_ions = aux[0]

junk4 = fromfile(fid, int32, 1)  # Read record start tag
r = fromfile(fid, single, int(junk4[0]/4)) # jmax*N_ions*3
junk5 = fromfile(fid, int32, 1)  # Read record stop tag

print(jmax,N_ions)
print(junk0[0],junk1[0],junk2[0],junk3[0],junk4[0],junk5[0])
print(junk4[0]/4)

74257 1024
4 4 4 4 43008000 43008000
10752000.0


In [242]:
snap_num = int(i_free__fly/real(i_snap_x+di_snap_x))
snap_time = []
for k in range(1,snap_num+1):
    snap_time.append(linspace(di_snap_x*k*dt,(di_snap_x*k+i_snap_x)*dt,i_snap_x)*1000)
print(shape(snap_time))

(7, 500)


In [243]:
snap_time

[array([5.        , 5.000501  , 5.001002  , 5.00150301, 5.00200401,
        5.00250501, 5.00300601, 5.00350701, 5.00400802, 5.00450902,
        5.00501002, 5.00551102, 5.00601202, 5.00651303, 5.00701403,
        5.00751503, 5.00801603, 5.00851703, 5.00901804, 5.00951904,
        5.01002004, 5.01052104, 5.01102204, 5.01152305, 5.01202405,
        5.01252505, 5.01302605, 5.01352705, 5.01402806, 5.01452906,
        5.01503006, 5.01553106, 5.01603206, 5.01653307, 5.01703407,
        5.01753507, 5.01803607, 5.01853707, 5.01903808, 5.01953908,
        5.02004008, 5.02054108, 5.02104208, 5.02154309, 5.02204409,
        5.02254509, 5.02304609, 5.02354709, 5.0240481 , 5.0245491 ,
        5.0250501 , 5.0255511 , 5.0260521 , 5.02655311, 5.02705411,
        5.02755511, 5.02805611, 5.02855711, 5.02905812, 5.02955912,
        5.03006012, 5.03056112, 5.03106212, 5.03156313, 5.03206413,
        5.03256513, 5.03306613, 5.03356713, 5.03406814, 5.03456914,
        5.03507014, 5.03557114, 5.03607214, 5.03

In [245]:
import matplotlib.patches as patches

In [259]:
# Get T and fluo PM  All SimuType
address_reuseinit = all_subdir[plot_point]
address = all_subdir[plot_point]
init_data = data0[plot_point][0]
evol_data = data6[plot_point][0]
print(init_data)
print(evol_data)

# onlyfiles = [f for f in listdir(evol_data) if isfile(join(evol_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles))
# onlyfiles_reuseinit = [f for f in listdir(init_data) if isfile(join(init_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles_reuseinit))

# ta, te, ti = plot_T_and_PM_InitQ_Evol_AfterCool(address,
#                                            flag_plot=1,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,46000))

fname = address[29:38]
plt.figure(fname).clf()
ax1 = subplot(111)
te_filter = []
for k in range(3):
    te_filter.append(savgol_filter(te[:,k], 25, 1))
    ax1.semilogy(ta*1e3,te_filter[k]) ; ax1.text(0,1e-4,'Filtered temperatures',fontsize = 10,ha='left')
    # ax1.semilogy(ta*1e3,te[:,k])
    ax1.grid()
    
ax1.set_xlabel('time [µs]')
ax1.set_ylabel('T [K]')
title(fname)
    
time_switch = i_free__fly_Lan *dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly_init)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='orange')
for k in range(snap_num):
    ax1.add_patch(
     patches.Rectangle(
        (snap_time[k][0]+time_switch*1e3, 1e-4),
        snap_time[k][-1]-snap_time[k][0],
        0.01,
        edgecolor = 'blue',
        facecolor = 'red',
        fill=True
     ) )
time_switch = (i_free__fly_Lan+i_laser_fly_init+i_laser_fly_quench+i_relax_fly)*dt
ax1.vlines(time_switch*1e3,2e-4,1e4,color='purple')

tight_layout()

# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

I:/Universite_Simus/20201201\DC07_RF09\Try00\Temp_SimuTypeQ_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG
I:/Universite_Simus/20201201\DC07_RF09\Try00\Temp_SimuType6_01_N01024_Vrf0059_Udc0.4000D+01V_D1.0_S1.0RFG


In [207]:
r_reshape = reshape(r,(N_ions,3,int(junk4[0]/4/3/N_ions)),order='F')
shape(r_reshape)

(1024, 3, 3500)

In [205]:
i_snap_x*floor(i_free__fly/real(i_snap_x+di_snap_x))*3*1024

10752000.0

In [214]:
figure('my_little_cloud').clf()
time = 
for k in range(1024):
    scatter(r_reshape[k,0,time]*1000,r_reshape[k,1,time]*1000)
plt.pause(1)
plt.clf()

array([[-3.7377726e-05, -1.0340276e-05, -1.4234838e-04],
       [ 2.1036723e-05,  2.9938297e-05,  6.6393121e-05],
       [ 4.2628635e-06, -2.8354054e-05,  1.3681798e-04],
       [ 1.6865499e-05, -3.2866599e-05, -6.5543332e-05],
       [ 2.2379498e-05, -2.7629736e-07,  1.1615225e-04]], dtype=float32)

In [225]:
figure('my_little_cloud').clf()
for time in range(0,2000,200):
    for k in range(1024):
        scatter(r_reshape[k,0,time]*1000,r_reshape[k,1,time]*1000)
    plt.pause(1)
    plt.clf()

3.4982388e-06